### 샘플 데이터 모델링

In [81]:
import json
import os
import pandas as pd
import numpy as np

In [138]:
from tensorflow import random # random seed 설정용
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, Flatten, MaxPool2D, Dropout, GlobalAveragePooling2D

In [161]:
path = 'New_sample/라벨링데이터/General/A/[apt]attend_008A/'
get_on = []
get_off = []
keys = []

for i in os.listdir(path):
    with open(path+i,'r') as f:
        #print(i)
        info = json.load(f)
        if len(info['annotations']) == 0: pass
        else: 
            annotations = info['annotations'][0]
            keypoints = annotations.get('keypoints')
            # for k in keypoints:
            #     if k <= 2:
            #         keypoints.remove(k)
            keys.append(keypoints)
            get_on.append(annotations.get('get_on'))
            get_off.append(annotations.get('get_off'))
            print(keypoints)

[1074, 1020, 2, 1111, 923, 2, 1151, 752, 2, 1266, 829, 2, 1180, 1012, 2, 1371, 1077, 0, 1193, 798, 2, 1248, 643, 2, 1334, 462, 2, 1320, 342, 2, 1102, 736, 2, 1148, 649, 2, 1225, 456, 2, 1442, 563, 2, 1407, 763, 2, 1334, 889, 2]
[1094, 918, 2, 1131, 802, 2, 1185, 643, 2, 1311, 694, 2, 1234, 873, 2, 1189, 979, 2, 1233, 687, 2, 1288, 559, 2, 1357, 382, 2, 1357, 303, 2, 1121, 634, 2, 1178, 542, 2, 1232, 392, 2, 1418, 454, 2, 1400, 604, 2, 1344, 700, 2]
[1085, 909, 2, 1118, 787, 2, 1174, 634, 2, 1291, 684, 2, 1214, 862, 2, 1187, 967, 2, 1226, 678, 2, 1281, 542, 2, 1345, 377, 2, 1334, 302, 2, 1100, 624, 2, 1158, 543, 2, 1230, 389, 2, 1415, 450, 2, 1392, 599, 2, 1336, 693, 2]
[1082, 908, 2, 1113, 786, 2, 1155, 627, 2, 1279, 683, 2, 1205, 861, 2, 1180, 966, 2, 1215, 674, 2, 1270, 540, 2, 1321, 366, 2, 1321, 300, 2, 1117, 632, 2, 1152, 541, 2, 1221, 387, 2, 1399, 448, 2, 1377, 597, 2, 1323, 692, 2]
[1071, 904, 2, 1102, 782, 2, 1144, 623, 2, 1271, 679, 2, 1196, 857, 2, 1170, 962, 2, 1209, 671, 2

In [163]:
keys = np.array(keys).reshape(-1,16,1,3)

In [164]:
keys.shape

(124, 16, 1, 3)

In [181]:
print(get_on)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [189]:
target = []

for i in range(len(get_on)):
    if get_on[i] == True: target.append(0)
    elif get_off[i] == True: target.append(1)
    elif (get_on[i] == False) and (get_off[i] == False): target.append(2)

---

In [191]:
target = np.array(target).reshape(-1,1)
target.shape

(124, 1)

In [215]:
target

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [0],
       [2],
    

In [192]:
# 8:2 비율
train_X, test_X, train_y, test_y = train_test_split(keys, target,
                                                  stratify=target,
                                                  test_size=0.2,
                                                  random_state=11)

In [193]:
train_X.shape, test_X.shape

((99, 16, 1, 3), (25, 16, 1, 3))

---

In [175]:
random.set_seed(11)

In [209]:
model=Sequential()
model.add(Conv2D(3, kernel_size=3, padding='same', input_shape=(16, 1, 3))) # 3차원
# 1차원으로 데이터 형태 변환 Layer => Flatten
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax')) # 1차원

In [210]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 16, 1, 3)          84        
                                                                 
 flatten_5 (Flatten)         (None, 48)                0         
                                                                 
 dense_10 (Dense)            (None, 64)                3136      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 3)                 195       
                                                                 
Total params: 3,415
Trainable params: 3,415
Non-trainable params: 0
_________________________________________________________________


In [211]:
# 검증 데이터에 대한 loss 값이 3번 연속 개선되지 않으면 학습 중지
stopCB = EarlyStopping(monitor='val_loss', patience=3)

In [212]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy')

In [213]:
hist = model.fit(train_X, train_y,
                 batch_size=36,
                 validation_split = 0.2,
                 epochs=20,
                 callbacks=[stopCB])

Epoch 1/20
3/3 [==============================] - 1s 77ms/step - loss: 167.8433 - accuracy: 0.2278 - val_loss: 32.4311 - val_accuracy: 0.4500
Epoch 2/20
3/3 [==============================] - 0s 11ms/step - loss: 108.1720 - accuracy: 0.3038 - val_loss: 4.8225 - val_accuracy: 0.7000
Epoch 3/20
3/3 [==============================] - 0s 11ms/step - loss: 72.3354 - accuracy: 0.4810 - val_loss: 0.0562 - val_accuracy: 0.9500
Epoch 4/20
3/3 [==============================] - 0s 13ms/step - loss: 36.3763 - accuracy: 0.6962 - val_loss: 0.7455 - val_accuracy: 0.9000
Epoch 5/20
3/3 [==============================] - 0s 11ms/step - loss: 36.8097 - accuracy: 0.6076 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
3/3 [==============================] - 0s 11ms/step - loss: 14.7430 - accuracy: 0.7848 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/20
3/3 [==============================] - 0s 12ms/step - loss: 13.0052 - accuracy: 0.8354 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch

In [214]:
model.evaluate(test_X, test_y)

1/1 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]